In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import logging
from helper import getModelDict
delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

c = 3e8

FORMAT = '%(levelname)s: %(message)s at %(asctime)s'
logging.basicConfig(format=FORMAT,datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()

Welcome to JupyROOT 6.30/06


In [2]:
defaultPars = {'figure.figsize': (5, 4),
               'lines.markersize' : 4,
               'axes.titlesize' : 13,
               'font.size' : 13,
               'axes.labelsize' : 16,
               'xtick.labelsize' : 13,
               'ytick.labelsize' : 13,
               'legend.fontsize' : 10,
               "text.usetex": True,
               "font.family": "sans-serif",
               "font.sans-serif": ["Helvetica"],
               'font.family':'Times New Roman', 
               'font.serif':'Times New Roman',
               "savefig.dpi" : 300,
               'contour.linewidth' : 2.0,
               'lines.linewidth' : 2.0,
               'axes.grid' : True,
               'grid.linewidth' : 0.5,
               'grid.color' : 'lightgray',
               'axes.axisbelow' : True
               }
plt.rcParams.update(defaultPars)

### Delphes (ROOT) file

In [3]:
file = './pp2chi0chi0J_test/Events/run_01/test.root'
c_light = 2.99792458e8

### Check single event

In [24]:
 
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")

# Filter event:
ievt = None
tmax = 0.0
iptc = None
npass_min = 10
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    npass = 0
    for ip,particle in enumerate(tree.ParticleProp):
        r_entry = np.array([particle.X,particle.Y,particle.Z])/1e3
        t_entry = particle.T
        rho_entry = np.linalg.norm(r_entry[:2])
        L_entry = np.linalg.norm(r_entry)
        t_light = L_entry/c_light
        t_readout = t_entry - t_light

        if t_readout > tmax:
            tmax = t_readout
            iptc = ip
        if t_readout > 25e-9  and (2.0 < rho_entry < 3.5):
            npass += 1
        if npass >= npass_min:
            print(f'EVENT FOUND:{i}, with {npass} particles satisfying criteria')
            ievt = i
            break
    if ievt is not None:
        break


# If not found, use first event
if ievt is None:
    print(f'EVENT NOT FOUND (tmax = {tmax:1.3e})')
    ievt = 0
else:
    print(f'Found particle: {particle.PID}')
    print(f'with Time readout = {t_readout:1.2e}')
    print(f'Rho, |z| (m)= {rho_entry:1.2e}, {np.abs(r_entry[2]):1.2e}')


tree.GetEntry(ievt)
# Single out a final (stable) particle from a LLP decay
print('\nBefore propagation')
print('  LLP decay position and momenta:')
illp = None
for daughter in tree.llpFinalDaughters:
    
    if illp is None:
        illp = daughter.M1
    elif illp == daughter.M1: # Do not repeat LLPs
        continue
    else:
        illp = daughter.M1

    llp = tree.llpParticles[illp]
    print(f'\t  LLP PDG = {llp.PID}')
    print(f'\t  Momentum (GeV)= ({llp.X:1.2e},{llp.Y:1.2e},{llp.Z:1.2e})')
    print(f'\t  Position (mm)= ({daughter.X:1.2e},{daughter.Y:1.2e},{daughter.Z:1.2e})')
    print(f'\t  Rho, |z| (m)= ({np.sqrt(daughter.X**2 + daughter.Y**2)/1e3:1.2e},{np.abs(daughter.Z)/1e3:1.2e})')
    print(f'\t  Time (s) = {daughter.T:1.4e}')
    print(f'\t  Time (mm) = {daughter.T*c_light*1e3:1.4e}\n')
        
        

# idaughter = 1
# illp = 0
# n = 0
# for daughter in tree.llpFinalDaughters:
#     if daughter.M1 != illp:
#         continue
#     n += 1
#     if n-1 != idaughter:
#         continue
#     print(daughter.PID,daughter.M1,daughter.E)
#     print(f'  Position (mm)= ({daughter.X:1.2e},{daughter.Y:1.2e},{daughter.Z:1.2e})')
#     print(f'  Rho, |z| (m)= ({np.sqrt(daughter.X**2 + daughter.Y**2)/1e3:1.2e},{np.abs(daughter.Z)/1e3:1.2e})')
#     print(f'  Time (s) = {daughter.T:1.4e}')
#     print(f'  Time (mm) = {daughter.T*c_light*1e3:1.4e}')
#     llp = tree.llpParticles[daughter.M1]
#     llpMom = np.array([llp.Px,llp.Py,llp.Pz,llp.E])
#     llp_beta = llpMom[:3]/llpMom[3]
#     llp_decay = c_light*llp_beta*daughter.T
#     print(llpMom)
#     print(llp.PID)
#     print(f' LLP decay position (m)= ({llp_decay[0]:1.2e},{llp_decay[1]:1.2e},{llp_decay[2]:1.2e})')
#     break

# # Compute time to propate to the ECAL (assuming that it reaches the barrel):
# r0 = np.array([daughter.X,daughter.Y,daughter.Z])
# t0 = daughter.T
# daughterMom = np.array([daughter.Px,daughter.Py,daughter.Pz,daughter.E])
# daughter_beta = daughterMom[:3]/daughterMom[3]
# vx = daughter_beta[0]*c_light
# vy = daughter_beta[1]*c_light
# x0 = r0[0]*1e-3
# y0 = r0[1]*1e-3
# R = 1.4
# t = t0 + (1.0/(2*(vx**2 + vy**2)))*(-2*vx*x0 - 2*vy*y0 +  np.sqrt((2*vx*x0 + 2*vy*y0)**2 - 4*(vx**2 + vy**2)*(-R**2 + x0**2 + y0**2)))
# print('\nPropagation to the barrel (manual):')
# print(f'Entry time (s) = {t:1.2e}')
# r_entry = r0*1e-3 + daughter_beta*c_light*(t-t0)
# print(f'Entry  Position (m)= ({r_entry[0]:1.2e},{r_entry[1]:1.2e},{r_entry[2]:1.2e})')

print('\nPropagation (Delphes)')
npass = 0
for particle in tree.ParticleProp:
    # if particle.PID != daughter.PID:
        # continue
    # if abs(particle.E-daughter.E)/daughter.E > 1e-3:
        # continue

    r_entry = np.array([particle.X,particle.Y,particle.Z])/1e3
    t_entry = particle.T
    L_entry = np.linalg.norm(r_entry)
    t_light = L_entry/c_light
    t_readout = t_entry - t_light
    rho = np.sqrt(particle.X**2 + particle.Y**2)/1e3

    if t_readout > 25e-9 and (2.0 < rho < 3.5):
        npass += 1
        if npass < 5:
            r = np.array([particle.X,particle.Y,particle.Z])
            rho = np.linalg.norm(r[:2])
            eta = np.atanh(r[2]/np.linalg.norm(r))
            phi = np.atan2(r[1],r[0])
            print(f'PID = {particle.PID}, E = {particle.E:1.2f}')
            print(f'  Position (mm)= ({r[0]:1.2e},{r[1]:1.2e},{r[2]:1.2e})')
            print(f'  Eta, phi = ({eta:1.2f},{phi:1.2f})')
            print(f'  Rho, |z| (m)= ({np.sqrt(particle.X**2 + particle.Y**2)/1e3:1.2e},{np.abs(particle.Z)/1e3:1.2e})')
            print(f'  Time = {particle.T:1.2e}')
            print(f'Readout time: {t_readout:1.2e}')
    # break
print(f'{npass} Particles passed')




# print('\nHadrons:')
# for particle in tree.CHadronsProp:
#     print(f'  Position (mm)= ({particle.X:1.2e},{particle.Y:1.2e},{particle.Z:1.2e})')
#     print(f'  Rho, |z| (m)= ({np.sqrt(particle.X**2 + particle.Y**2)/1e3:1.2e},{np.abs(particle.Z)/1e3:1.2e})')
#     print(f'  Time = {particle.T:1.2e}')
#     break

# print('\nTracks:')
# for particle in tree.Tracks:
#     print(f'  Position (mm)= ({particle.X:1.2e},{particle.Y:1.2e},{particle.Z:1.2e})')
#     print(f'  Position() (mm)= ({particle.X:1.2e},{particle.Y:1.2e},{particle.Z:1.2e})')
#     print(f'  Rho, |z| (m)= ({np.sqrt(particle.X**2 + particle.Y**2)/1e3:1.2e},{np.abs(particle.Z)/1e3:1.2e})')
#     print(f'  Time = {particle.T:1.2e}')
#     break

print('\nOnTime Calorimeter:')

tmin = 1e10
tmax = 0.0
for tower in tree.TowerOnTime:
    t = tower.T
    r = np.array([tower.X,tower.Y,tower.Z])
    eta = tower.Eta
    phi = tower.Phi
    t_readout = t- np.linalg.norm(r)/c_light
    tmin = min(tmin,t_readout)
    tmax = max(tmax,t_readout)
    # print('eta=',eta,'phi=',phi)
    # print('r = ',r,'E=',tower.P4().E(),'Eem=',tower.Eem,'Ehad=',tower.Ehad)
    # print(f'E = {tower.E:1.2f}, t(s) = {t:1.2e}, t(readout)(s) = {t_readout:1.2e}')
    continue
    for particle in tower.Particles:
        # print(particle.PID,particle.E)
        for p in tree.ParticleProp:
            if particle.PID != p.PID:
                continue
            if abs(particle.E-p.E)/p.E > 1e-3:
                continue
            r_entry = np.array([p.X,p.Y,p.Z])/1e3
            t_entry = p.T
            L_entry = np.linalg.norm(r_entry)
            t_light = L_entry/c_light
            t_readout = t_entry - t_light
            rho = np.sqrt(p.X**2 + p.Y**2)/1e3

            # if t_readout < 1e-9:
                # continue
            print(f'  PID = {p.PID}, E = {p.E:1.2e}')
            # print(f'  Position (mm)= ({p.X:1.2e},{p.Y:1.2e},{p.Z:1.2e})')
            print(f'  Rho, |z| (m)= ({rho:1.2e},{np.abs(p.Z)/1e3:1.2e})')
            print(f'  Readout Time = {t_readout:1.2e}')
    break
print(f'{tmin:1.2e} s < Readout Time < {tmax:1.2e} s')



print('\nDelayed ECAL:')
tmin = 1e10
tmax = 0.0
for tower in tree.ETowerDelayed:
    t = tower.T
    r = np.array([tower.X,tower.Y,tower.Z])
    eta = tower.Eta
    phi = tower.Phi
    t_readout = t- np.linalg.norm(r)/c_light
    tmin = min(tmin,t_readout)
    tmax = max(tmax,t_readout)
    print(f'\t  # hits = {tower.NTimeHits}')
    print(f'\t  E = {tower.E:1.2f}, Eta = {tower.Eta:1.2f}, Phi = {tower.Phi:1.2f}, t(s) = {tower.T:1.2e}, t(readout)(s) = {tower.T:1.2e}')
    # break
print(f'{tmin:1.2e} s < Readout Time < {tmax:1.2e} s')

print('\nDelayed HCAL:')
tmin = 1e10
tmax = 0.0
for tower in tree.HTowerDelayed:
    t = tower.T
    r = np.array([tower.X,tower.Y,tower.Z])
    eta = tower.Eta
    phi = tower.Phi
    t_readout = t- np.linalg.norm(r)/c_light
    tmin = min(tmin,t_readout)
    tmax = max(tmax,t_readout)
    print(f'\t  # hits = {tower.NTimeHits}')
    print(f'\t  E = {tower.E:1.2f}, Eta = {tower.Eta:1.2f}, Phi = {tower.Phi:1.2f}, t(s) = {tower.T:1.2e}, t(readout)(s) = {tower.T:1.2e}')
    # break
print(f'{tmin:1.2e} s < Readout Time < {tmax:1.2e} s')


print('\nDelayed Calorimeter:')
tmin = 1e10
tmax = 0.0
for tower in tree.TowerDelayed:
    t = tower.T
    r = np.array([tower.X,tower.Y,tower.Z])
    eta = tower.Eta
    phi = tower.Phi
    t_readout = t- np.linalg.norm(r)/c_light
    tmin = min(tmin,t_readout)
    tmax = max(tmax,t_readout)
    print(f'\t  # hits = {tower.NTimeHits}')
    print(f'\t  E = {tower.E:1.2f}, Eta = {tower.Eta:1.2f}, Phi = {tower.Phi:1.2f}, t(s) = {tower.T:1.2e}, t(readout)(s) = {tower.T:1.2e}')
    # break
print(f'{tmin:1.2e} s < Readout Time < {tmax:1.2e} s')

f.Close()

EVENT FOUND:21, with 10 particles satisfying criteria
Found particle: 22
with Time readout = 2.97e-08
Rho, |z| (m)= 2.18e+00, 2.30e+00

Before propagation
  LLP decay position and momenta:
	  LLP PDG = 4000023
	  Momentum (GeV)= (8.81e-16,-5.97e-16,-1.83e-15)
	  Position (mm)= (2.12e+03,-1.64e+03,-6.93e+02)
	  Rho, |z| (m)= (2.68e+00,6.93e-01)
	  Time (s) = 2.6861e-08
	  Time (mm) = 8.0526e+03

	  LLP PDG = -4000023
	  Momentum (GeV)= (8.81e-16,-5.97e-16,-1.83e-15)
	  Position (mm)= (-1.64e+03,1.43e+03,-2.30e+03)
	  Rho, |z| (m)= (2.18e+00,2.30e+00)
	  Time (s) = 4.0265e-08
	  Time (mm) = 1.2071e+04


Propagation (Delphes)
PID = -211, E = 5.88
  Position (mm)= (-1.64e+03,1.43e+03,-2.30e+03)
  Eta, phi = (-0.92,2.43)
  Rho, |z| (m)= (2.18e+00,2.30e+00)
  Time = 4.03e-08
Readout time: 2.97e-08
PID = 2112, E = 2.04
  Position (mm)= (-1.64e+03,1.43e+03,-2.30e+03)
  Eta, phi = (-0.92,2.43)
  Rho, |z| (m)= (2.18e+00,2.30e+00)
  Time = 4.03e-08
Readout time: 2.97e-08
PID = -2112, E = 1.00
  P